<a href="https://colab.research.google.com/github/brochowski/ekantorwidget/blob/main/brochowski_excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import sklearn

sklearn.__version__

'1.2.2'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Bufor/export.csv', sep=';')
dta_cp = df.copy()
dta_cp.columns = dta_cp.iloc[0]
dta_cp = dta_cp.iloc[1:]
dta_cp = dta_cp.iloc[:, 2:]

new_column_names = ['DATE', 'EX_BUYING', 'EX_SELLING', 'SPREAD']
dta_cp.columns = new_column_names

dta_cp['EX_BUYING'] = dta_cp['EX_BUYING'].str.replace(',', '.').astype(float)  # Konwertuj kolumnę EX_BUYING na float
dta_cp['EX_SELLING'] = dta_cp['EX_SELLING'].str.replace(',', '.').astype(float)  # Konwertuj kolumnę EX_SEELING na float
dta_cp['DATE'] = dta_cp['DATE'].apply(lambda x: x.split('.')[0])

dta_cp['SPREAD'] = dta_cp['EX_SELLING'] - dta_cp['EX_BUYING']


from datetime import datetime

def konwertuj_na_date(data_str):
    maska = "%Y-%m-%dT%H:%M:%S"
    data = datetime.strptime(data_str, maska)
    return data


dta_cp['DATE'] = dta_cp['DATE'].apply(konwertuj_na_date)

dta_cp['DAY_OF_WEEK'] = dta_cp['DATE'].apply(lambda x: x.weekday())
dta_cp['HOUR_NO'] = dta_cp['DATE'].apply(lambda x: int(x.strftime('%H')))


##print(dta_cp.columns)

print(dta_cp.head())
dta_cp.info()

                 DATE  EX_BUYING  EX_SELLING  SPREAD  DAY_OF_WEEK  HOUR_NO
2 2023-06-15 23:13:05     4.0380      4.1036  0.0656            3       23
3 2023-06-15 23:14:24     4.0375      4.1046  0.0671            3       23
4 2023-06-15 23:16:48     4.0382      4.1009  0.0627            3       23
5 2023-06-15 23:16:55     4.0383      4.1020  0.0637            3       23
6 2023-06-15 23:26:48     4.0385      4.1008  0.0623            3       23
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140 entries, 2 to 2141
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DATE         2140 non-null   datetime64[ns]
 1   EX_BUYING    2140 non-null   float64       
 2   EX_SELLING   2140 non-null   float64       
 3   SPREAD       2140 non-null   float64       
 4   DAY_OF_WEEK  2140 non-null   int64         
 5   HOUR_NO      2140 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2)
memo

In [ ]:
dta_cp['SPREAD_CAT'] = pd.cut(x=dta_cp.SPREAD, bins=3, labels=['small','medium','high'])
dta_cp

,DATE,EX_BUYING,EX_SELLING,SPREAD,DAY_OF_WEEK,HOUR_NO,SPREAD_CAT
2,2023-06-15 23:13:05,4.0380,4.1036,0.0656,3,23,small
3,2023-06-15 23:14:24,4.0375,4.1046,0.0671,3,23,medium
4,2023-06-15 23:16:48,4.0382,4.1009,0.0627,3,23,small
5,2023-06-15 23:16:55,4.0383,4.1020,0.0637,3,23,small
6,2023-06-15 23:26:48,4.0385,4.1008,0.0623,3,23,small
...,...,...,...,...,...,...,...
2137,2023-07-11 05:49:35,4.0046,4.0609,0.0563,1,5,small
2138,2023-07-11 06:05:04,4.0040,4.0604,0.0564,1,6,small
2139,2023-07-11 06:20:09,4.0043,4.0607,0.0564,1,6,small
2140,2023-07-11 06:35:13,4.0034,4.0599,0.0565,1,6,small


In [ ]:
import statsmodels.api as sm
def get_trend_label(value):
    if value > 0:
        return 'increase'
    elif value < 0:
        return 'decrease'
    else:
        return 'stable'

dta_cp['trend_selling'] = np.nan

for i, row in dta_cp.iterrows():
    x = i  # Indeks wiersza
    y = row['EX_SELLING']
    X = sm.add_constant([x])  # Dodajemy stałą kolumnę X dla intercept'u
    model = sm.OLS([y], X)
    results = model.fit()
    slope = results.params[0]
    dta_cp.at[i, 'trend_selling'] = get_trend_label(slope)
    #dta_cp.at[i, 'trend_selling'] = slope

print(dta_cp)

                    DATE  EX_BUYING  EX_SELLING    SPREAD  DAY_OF_WEEK  \
2    2023-06-15 23:13:05  -0.314517    0.057014  0.444796            3   
3    2023-06-15 23:14:24  -0.341171    0.099708  0.537008            3   
4    2023-06-15 23:16:48  -0.303855   -0.058259  0.266519            3   
5    2023-06-15 23:16:55  -0.298525   -0.011296  0.327994            3   
6    2023-06-15 23:26:48  -0.287863   -0.062528  0.241929            3   
...                  ...        ...         ...       ...          ...   
2137 2023-07-11 05:49:35  -2.095003   -1.766012 -0.126920            1   
2138 2023-07-11 06:05:04  -2.126988   -1.787358 -0.120772            1   
2139 2023-07-11 06:20:09  -2.110996   -1.774550 -0.120772            1   
2140 2023-07-11 06:35:13  -2.158973   -1.808705 -0.114625            1   
2141 2023-07-11 06:36:53  -2.180296   -1.830052 -0.120772            1   

      HOUR_NO SPREAD_CAT trend_selling  
2          23      small      increase  
3          23     medium     

In [ ]:
dta_cp.describe()

,EX_BUYING,EX_SELLING,SPREAD,DAY_OF_WEEK,HOUR_NO
count,2.140000e+03,2.140000e+03,2.140000e+03,2140.000000,2140.000000
mean,-3.968414e-14,1.154798e-14,1.394523e-16,2.981776,11.735981
std,1.000234e+00,1.000234e+00,1.000234e+00,2.076414,6.828488
min,-2.180296e+00,-2.551578e+00,-1.307235e+00,0.000000,0.000000
25%,-5.010949e-01,-5.449686e-01,-2.498691e-01,1.000000,6.000000
50%,-2.185627e-01,-2.546506e-01,-1.207721e-01,3.000000,12.000000
75%,3.278440e-01,3.910933e-01,-3.970006e-03,5.000000,18.000000
max,3.667588e+00,3.929343e+00,3.985742e+00,6.000000,23.000000


In [ ]:
dta_cp.describe().T

,count,mean,std,min,25%,50%,75%,max
EX_BUYING,2140.0,4.043900,0.018763,4.0030,4.0345,4.0398,4.050050,4.1127
EX_SELLING,2140.0,4.102265,0.023428,4.0425,4.0895,4.0963,4.111425,4.1943
SPREAD,2140.0,0.058365,0.016271,0.0371,0.0543,0.0564,0.058300,0.1232
DAY_OF_WEEK,2140.0,2.981776,2.076414,0.0000,1.0000,3.0000,5.000000,6.0000
HOUR_NO,2140.0,11.735981,6.828488,0.0000,6.0000,12.0000,18.000000,23.0000


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dta_cp[['EX_BUYING', 'EX_SELLING', 'SPREAD']] = scaler.fit_transform(dta_cp[['EX_BUYING', 'EX_SELLING', 'SPREAD']])
dta_cp

,DATE,EX_BUYING,EX_SELLING,SPREAD,DAY_OF_WEEK,HOUR_NO,SPREAD_CAT,trend_selling
2,2023-06-15 23:13:05,-0.314517,0.057014,0.444796,3,23,small,increase
3,2023-06-15 23:14:24,-0.341171,0.099708,0.537008,3,23,medium,increase
4,2023-06-15 23:16:48,-0.303855,-0.058259,0.266519,3,23,small,increase
5,2023-06-15 23:16:55,-0.298525,-0.011296,0.327994,3,23,small,increase
6,2023-06-15 23:26:48,-0.287863,-0.062528,0.241929,3,23,small,increase
...,...,...,...,...,...,...,...,...
2137,2023-07-11 05:49:35,-2.095003,-1.766012,-0.126920,1,5,small,increase
2138,2023-07-11 06:05:04,-2.126988,-1.787358,-0.120772,1,6,small,increase
2139,2023-07-11 06:20:09,-2.110996,-1.774550,-0.120772,1,6,small,increase
2140,2023-07-11 06:35:13,-2.158973,-1.808705,-0.114625,1,6,small,increase


In [ ]:
import plotly.express as px

df_plotly = dta_cp.reset_index()
px.line(dta_cp, 'DATE', 'EX_BUYING', width=1000, height=500, title='WYKRESIK_1')

In [ ]:
px.line(dta_cp, 'DATE', 'EX_SELLING', width=1000, height=500, title='WYKRESIK_2')

In [ ]:
px.line(dta_cp, 'DATE', 'SPREAD', width=1000, height=500, title='WYKRESIK_3')

In [ ]:
px.line(dta_cp, 'DATE', 'SPREAD_CAT', width=1000, height=500, title='WYKRESIK_4')

In [ ]:
px.line(dta_cp, 'DATE', 'trend_selling', width=1000, height=500, title='WYKRESIK_5')

In [ ]:
dummy_spread = pd.get_dummies(dta_cp['SPREAD_CAT'],drop_first=True, prefix='spread')
dta_cp = pd.concat([dta_cp, dummy_spread], axis=1)

In [ ]:
dummy_trend = pd.get_dummies(dta_cp['trend_selling'],drop_first=True, prefix='selling_trend')
dta_cp = pd.concat([dta_cp, dummy_trend], axis=1)

In [ ]:
dta_cp = dta_cp.drop('SPREAD_CAT', axis=1)
dta_cp = dta_cp.drop('trend_selling', axis=1)
dta_cp

,DATE,EX_BUYING,EX_SELLING,SPREAD,DAY_OF_WEEK,HOUR_NO,spread_medium,spread_high,selling_trend_increase
2,2023-06-15 23:13:05,-0.314517,0.057014,0.444796,3,23,0,0,1
3,2023-06-15 23:14:24,-0.341171,0.099708,0.537008,3,23,1,0,1
4,2023-06-15 23:16:48,-0.303855,-0.058259,0.266519,3,23,0,0,0
5,2023-06-15 23:16:55,-0.298525,-0.011296,0.327994,3,23,0,0,0
6,2023-06-15 23:26:48,-0.287863,-0.062528,0.241929,3,23,0,0,0
...,...,...,...,...,...,...,...,...,...
2137,2023-07-11 05:49:35,-2.095003,-1.766012,-0.126920,1,5,0,0,0
2138,2023-07-11 06:05:04,-2.126988,-1.787358,-0.120772,1,6,0,0,0
2139,2023-07-11 06:20:09,-2.110996,-1.774550,-0.120772,1,6,0,0,0
2140,2023-07-11 06:35:13,-2.158973,-1.808705,-0.114625,1,6,0,0,0
